In [1]:
import torch
from torch.utils.data import DataLoader
from cellshape_cloud.vendor.chamfer_distance import ChamferLoss
from cellshape_cloud.pointcloud_dataset import (
    PointCloudDataset,
    SingleCellDataset,
)
from cellshape_cloud.cloud_autoencoder import CloudAutoEncoder

path = "/home/mvries/Documents/CellShape/UploadData/cellshapeOutput/Models/cloud_autoencoder/dgcnn_foldinget_50_004_predictions_NEW.pt"

/home/mvries/anaconda3/envs/cellshape-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = CloudAutoEncoder(
    num_features=50, k=20, encoder_type="dgcnn", decoder_type="foldingnet"
)
checkpoint = torch.load(path)
model_dict = model.state_dict()  # load parameters from pre-trained FoldingNet

model.load_state_dict(checkpoint["model_state_dict"])
checkpoint["loss"]

54012.08462524414

In [3]:
root_dir = "/home/mvries/Documents/CellShape/UploadData/cellshapeData/"
df = "/home/mvries/Documents/CellShape/UploadData/cellshapeData/all_cell_data.csv"
dataset = SingleCellDataset(df, root_dir, cell_component="cell")

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [4]:
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
all_feat_cell = []
all_outputs = []
all_inputs = []
labels_cell = []
serial_numbers_cell = []
for data in tqdm(dataloader):
    inputs = data[0]
    lab = data[1]
    ser_num = data[3]
    inputs = inputs.to(device)
    batch_size = inputs.shape[0]

    output, features = model(inputs)
    all_inputs.append(torch.squeeze(inputs).detach().cpu().numpy())
    all_outputs.append(torch.squeeze(output).detach().cpu().numpy())
    all_feat_cell.append(torch.squeeze(features).detach().cpu().numpy())
    labels_cell.append(lab[0])
    serial_numbers_cell.append(ser_num[0])

100%|█████████████████████████████████████| 70167/70167 [12:50<00:00, 91.10it/s]


In [5]:
dataset = SingleCellDataset(df, root_dir, cell_component="nuc")
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
all_feat_nuc = []
all_outputs = []
all_inputs = []
labels_nuc = []
serial_numbers_nuc = []
for data in tqdm(dataloader):
    inputs = data[0]
    lab = data[1]
    ser_num = data[3]
    inputs = inputs.to(device)
    batch_size = inputs.shape[0]

    output, features = model(inputs)
    all_inputs.append(torch.squeeze(inputs).detach().cpu().numpy())
    all_outputs.append(torch.squeeze(output).detach().cpu().numpy())
    all_feat_nuc.append(torch.squeeze(features).detach().cpu().numpy())
    labels_nuc.append(lab[0])
    serial_numbers_nuc.append(ser_num[0])

  0%|                                                 | 0/70167 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/mvries/Documents/CellShape/UploadData/cellshapeData/Plate1/stacked_pointcloud_nucleus/Palbociclib/0001_0001_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.ply'

In [ ]:
# Need to join cell and nuc features and then do predictions on those